In [1]:
import urllib
import urllib.request
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [148]:
binance_base = "https://<>.binance.com"
binance_subdomains = ["api", "api1", "api2", "api3"]

binance_url = binance_base.replace('<>', binance_subdomains[0])

binance_endpoints = {
    'ping': ('GET', '/api/v3/ping'),
    'server_time': ('GET', '/api/v3/time'),
    'exchange_info': ('GET', '/api/v3/exchangeInfo'),
    'order_book': ('GET', '/api/v3/depth', {'symbol': True, 'limit': False}),
    'recent_trades': ('GET', '/api/v3/trades', {'symbol': True, 'limit': False}),
    'average_price': ('GET', '/api/v3/avgPrice', {'symbol': True}),
    'price': ('GET', '/api/v3/ticker/price', {'symbol': False}),
    'best_book_price': ('GET', '/api/v3/ticker/bookTicker', {'symbol': False})
}

In [149]:
def get_price(symbol):
    binance_uri = binance_url + binance_endpoints['average_price'][1] + f'?symbol={symbol}'
    web_url = urllib.request.urlopen(binance_uri)
    data = web_url.read()
    encoding = web_url.info().get_content_charset('utf-8')
    JSON_object = json.loads(data.decode(encoding))
    return {"symbol": symbol, "price": float(JSON_object["price"])}

def get_symbols_prices(symbols):
    prices = []
    for s in symbols:
        prices.append(get_price(s))
    return prices

In [150]:
binance_uri = binance_url + binance_endpoints['exchange_info'][1]

web_url = urllib.request.urlopen(binance_uri)
data = web_url.read()
encoding = web_url.info().get_content_charset('utf-8')
JSON_object = json.loads(data.decode(encoding))

df_symbols = pd.DataFrame(JSON_object['symbols'])
df_symbols = df_symbols.set_index("symbol")
df_symbols.head()

,status,baseAsset,baseAssetPrecision,quoteAsset,quotePrecision,quoteAssetPrecision,baseCommissionPrecision,quoteCommissionPrecision,orderTypes,icebergAllowed,ocoAllowed,quoteOrderQtyMarketAllowed,isSpotTradingAllowed,isMarginTradingAllowed,filters,permissions
symbol,,,,,,,,,,,,,,,,
1INCHBTC,TRADING,1INCH,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
1INCHBUSD,TRADING,1INCH,8,BUSD,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[SPOT]
1INCHDOWNUSDT,TRADING,1INCHDOWN,8,USDT,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,False,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[LEVERAGED]
1INCHUPUSDT,TRADING,1INCHUP,8,USDT,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,False,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[LEVERAGED]
1INCHUSDT,TRADING,1INCH,8,USDT,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"


In [160]:
selected_symbols = ["BNBEUR", "ADAUSDT", "BTCEUR", "BNBETH", "BNBBTC", "ETHBTC", "BNBUSDT", 
                    "ADABTC", "ADAETH", "ADABNB", "DOGEBTC", "ETHUSDT", "EURUSDT", "ADAEUR", 
                    "ETHEUR", "BTCUSDT", "DOGEEUR", "DOGEUSDT"]
df_symbols = df_symbols.loc[selected_symbols]
df_symbols = df_symbols.sort_index()

In [161]:
prices = get_symbols_prices(selected_symbols)

In [162]:
df_symbols["price"] = pd.DataFrame().from_dict(prices).set_index("symbol")

In [163]:
#df_symbols["f_PRICE_minPrice"] = df_symbols["filters"].apply(lambda f: [float(x["minPrice"]) for x in f if x["filterType"]=="PRICE_FILTER"][0])
#df_symbols["f_PRICE_tickSize"] = df_symbols["filters"].apply(lambda f: [float(x["tickSize"]) for x in f if x["filterType"]=="PRICE_FILTER"][0])
df_symbols["f_LOT_minQty"] = df_symbols["filters"].apply(lambda f: [float(x["minQty"]) for x in f if x["filterType"]=="LOT_SIZE"][0])
df_symbols["f_LOT_stepSize"] = df_symbols["filters"].apply(lambda f: [float(x["stepSize"]) for x in f if x["filterType"]=="LOT_SIZE"][0])
df_symbols["f_NOTIONA_minNotional"] = df_symbols["filters"].apply(lambda f: [float(x["minNotional"]) for x in f if x["filterType"]=="MIN_NOTIONAL"][0])
df_symbols["f_NOTIONA_applyToMarket"] = df_symbols["filters"].apply(lambda f: [x["applyToMarket"] for x in f if x["filterType"]=="MIN_NOTIONAL"][0])
df_symbols["f_LOT_step_roundto"] = df_symbols["f_LOT_stepSize"].apply(lambda x: int(np.absolute(np.log10(x))))

In [164]:
f_cols = [c for c in df_symbols.columns if c[:2]=="f_"]

In [165]:
base_eur_map = df_symbols[df_symbols["quoteAsset"]=="EUR"][["baseAsset", "price"]].set_index("baseAsset").to_dict()["price"]
df_symbols["eur_price"] = df_symbols["baseAsset"].map(base_eur_map)

In [166]:
def my_ceil(a, ndigits=0):
    return np.true_divide(np.ceil(a * 10**ndigits), 10**ndigits)

In [167]:
df_symbols["min_qty"] = df_symbols["f_NOTIONA_minNotional"] / df_symbols["price"]

df_symbols["min_qty_rounded"] = df_symbols.apply(lambda x: my_ceil(x["min_qty"], ndigits=x["f_LOT_step_roundto"]), axis=1)
df_symbols["min_qty_val_notional"] = df_symbols["min_qty_rounded"] * df_symbols["price"]
df_symbols["min_qty_val_in_eur"] = df_symbols["min_qty_rounded"] * df_symbols["eur_price"]

min_qty_map = df_symbols.groupby("baseAsset")["min_qty_rounded"].max().to_dict()
df_symbols["min_safe_qty"] = df_symbols["baseAsset"].map(min_qty_map)
df_symbols["min_safe_qty_val_in_eur"] =  df_symbols["min_safe_qty"] * df_symbols["eur_price"]

In [168]:
df_symbols[["min_safe_qty", "min_safe_qty_val_in_eur", "f_LOT_minQty", "f_LOT_stepSize", "f_NOTIONA_minNotional", "min_qty_val_notional", "price", "min_qty", "min_qty_rounded", "min_qty_val_in_eur"]]

,min_safe_qty,min_safe_qty_val_in_eur,f_LOT_minQty,f_LOT_stepSize,f_NOTIONA_minNotional,min_qty_val_notional,price,min_qty,min_qty_rounded,min_qty_val_in_eur
symbol,,,,,,,,,,
ADABNB,20.000000,27.970107,1.000000,1.000000,0.0500,0.052214,0.002611,19.152098,20.000000,27.970107
ADABTC,20.000000,27.970107,1.000000,1.000000,0.0001,0.000123,0.000031,3.241491,4.000000,5.594021
ADAETH,20.000000,27.970107,1.000000,1.000000,0.0050,0.005208,0.000434,11.519941,12.000000,16.782064
ADAEUR,20.000000,27.970107,0.010000,0.010000,10.0000,10.013298,1.398505,7.150491,7.160000,10.013298
ADAUSDT,20.000000,27.970107,0.010000,0.010000,10.0000,10.001662,1.698075,5.889021,5.890000,8.237197
BNBBTC,0.031000,16.624490,0.010000,0.010000,0.0001,0.000118,0.011815,0.008464,0.010000,5.362739
BNBETH,0.031000,16.624490,0.001000,0.001000,0.0050,0.005153,0.166221,0.030080,0.031000,16.624490
BNBEUR,0.031000,16.624490,0.000100,0.000100,10.0000,10.028321,536.273874,0.018647,0.018700,10.028321
BNBUSDT,0.031000,16.624490,0.000100,0.000100,10.0000,10.014133,650.268383,0.015378,0.015400,8.258618
